In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency




# 1. Cardinalidad y tipo de variable

In [ ]:
# Función que calcula la cardinalidad y el tipo de variable
def cardinalidad(df_in, umbral_categoria = 10, umbral_continua = 30):
    resultado = []
    for col in df_in.columns:
        card = df_in[col].nunique()
        porc_card = df_in[col].nunique()/len(df_in) * 100
        tipo = df_in[col].dtypes
        if card == 2:
            clasif = "Binaria"
        elif card < umbral_categoria:
            clasif = "Categórica"
        elif card >= umbral_categoria:
            if porc_card >= umbral_continua:
                clasif = "Numérica Continua"
            else:
                clasif = "Numérica Discreta"

        # Añadimos los resultados a la lista
        resultado.append({
            "columna": col,
            "tipo": tipo,
            "cardinalidad": card,
            "porcentaje_card": porc_card,
            "clasificacion": clasif
        })
    
    # Convertimos la lista de dicts en dataframe
    df_resul = pd.DataFrame(resultado)
        
    return df_resul

# 2. Frecuencias absolutas y relativas para variables categóricas

In [ ]:
# Para las variables categóricas tengo varias opciones

# 1. función para ver frecuencias absolutas
def frec_abs(df, columnas_categoricas):
    for catego in columnas_categoricas:
        print(f"Para {catego}")
        print(df[catego].value_counts())
        print("\n")

# 2. función para ver frecuencias relativas
def frec_rel(df, columnas_categoricas):
    for catego in columnas_categoricas:
        print(f"Para {catego}")
        print(df[catego].value_counts()/len(df)*100)
        print("\n")


# 3. Función que representa las frecuencias absolutas o relativas en gráficos
def pinta_distribucion_categoricas(df, columnas_categoricas, relativa=False, mostrar_valores=False):
    num_columnas = len(columnas_categoricas)
    num_filas = (num_columnas // 2) + (num_columnas % 2)

    fig, axes = plt.subplots(num_filas, 2, figsize=(15, 5 * num_filas))
    axes = axes.flatten() 

    for i, col in enumerate(columnas_categoricas):
        ax = axes[i]
        if relativa:
            total = df[col].value_counts().sum()
            serie = df[col].value_counts().apply(lambda x: x / total)
            sns.barplot(x=serie.index, y=serie, ax=ax, palette='viridis', hue = serie.index, legend = False)
            ax.set_ylabel('Frecuencia Relativa')
        else:
            serie = df[col].value_counts()
            sns.barplot(x=serie.index, y=serie, ax=ax, palette='viridis', hue = serie.index, legend = False)
            ax.set_ylabel('Frecuencia')

        ax.set_title(f'Distribución de {col}')
        ax.set_xlabel('')
        ax.tick_params(axis='x', rotation=45)

        if mostrar_valores:
            for p in ax.patches:
                height = p.get_height()
                ax.annotate(f'{height:.2f}', (p.get_x() + p.get_width() / 2., height), 
                            ha='center', va='center', xytext=(0, 9), textcoords='offset points')

    for j in range(i + 1, num_filas * 2):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()



# 3. Medidas de posición para variables numéricas

In [ ]:
# Cómo quedarme con las columnas numéricas
def columnas_numericas(df):
    num_col = df.describe().columns.to_list()
    return num_col


In [ ]:

# Función para representar boxplots de columnas numéricas
def plot_multiple_boxplots(df, columns, dim_matriz_visual = 2):
    num_cols = len(columns)
    num_rows = num_cols // dim_matriz_visual + num_cols % dim_matriz_visual
    fig, axes = plt.subplots(num_rows, dim_matriz_visual, figsize=(12, 6 * num_rows))
    axes = axes.flatten()

    for i, column in enumerate(columns):
        if df[column].dtype in ['int64', 'float64']:
            sns.boxplot(data=df, x=column, ax=axes[i])
            axes[i].set_title(column)

    # Ocultar ejes vacíos
    for j in range(i+1, num_rows * 2):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()


# Función para crear boxplots de una variable agrupada con respecto a otra
def plot_boxplot_grouped(df, column_to_plot, group_column):
    if df[column_to_plot].dtype in ['int64', 'float64'] and df[group_column].dtype in ['object', 'category']:
        sns.boxplot(data=df, x=group_column, y=column_to_plot)
        plt.show()

# Función para calcular el rango intercuartílico
def get_IQR(df, col):
    return df[col].quantile(0.75) - df[col].quantile(0.25)

# 4. Medidas de dispersión variables numéricas

In [ ]:
# Función para calcular coeficiente de variación
def variabilidad(df):
    df_var = df.describe().loc[["std", "mean"]].T
    df_var["CV"] = df_var["std"]/df_var["mean"]
    return df_var

# Histgramas de la función de densidad de probabilidad (solo numéricas continuas)
def plot_histo_dens(df, columns, bins=None):
    num_cols = len(columns)
    num_rows = num_cols // 2 + num_cols % 2
    fig, axes = plt.subplots(num_rows, 2, figsize=(12, 6 * num_rows))
    axes = axes.flatten()

    for i, column in enumerate(columns):
        if df[column].dtype in ['int64', 'float64']:
            if bins:
                sns.histplot(df[column], kde=True, ax=axes[i], bins=bins)
            else:
                sns.histplot(df[column], kde=True, ax=axes[i])
            axes[i].set_title(f'Histograma y KDE de {column}')

    # Ocultar ejes vacíos
    for j in range(i + 1, num_rows * 2):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# 5. Outliers
## Usando la función siguiente me ahorro hacer lo de los dos apartados anteriores porque lo hace junto

In [ ]:
# Función que representa el histograma y el boxplot de varias variables
def plot_combined_graphs(df, columns, whisker_width=1.5):
    num_cols = len(columns)
    if num_cols:
        
        fig, axes = plt.subplots(num_cols, 2, figsize=(12, 5 * num_cols))
        print(axes.shape)

        for i, column in enumerate(columns):
            if df[column].dtype in ['int64', 'float64']:
                # Histograma y KDE
                sns.histplot(df[column], kde=True, ax=axes[i,0] if num_cols > 1 else axes[0])
                if num_cols > 1:
                    axes[i,0].set_title(f'Histograma y KDE de {column}')
                else:
                    axes[0].set_title(f'Histograma y KDE de {column}')

                # Boxplot
                sns.boxplot(x=df[column], ax=axes[i,1] if num_cols > 1 else axes[1], whis=whisker_width)
                if num_cols > 1:
                    axes[i,1].set_title(f'Boxplot de {column}')
                else:
                    axes[1].set_title(f'Boxplot de {column}')

        plt.tight_layout()
        plt.show()

# 6. Análisis multivariante

## 6.1. Análisis bivariante categóricas y numéricas

### 6.1.1. Análisis bivariante categórica

In [ ]:
# Función análisis bivariante categóricas
def plot_categorical_relationship_fin(df, cat_col1, cat_col2, relative_freq=False, show_values=False, size_group = 5):
    # Prepara los datos
    count_data = df.groupby([cat_col1, cat_col2]).size().reset_index(name='count')
    total_counts = df[cat_col1].value_counts()
    
    # Convierte a frecuencias relativas si se solicita
    if relative_freq:
        count_data['count'] = count_data.apply(lambda x: x['count'] / total_counts[x[cat_col1]], axis=1)

    # Si hay más de size_group categorías en cat_col1, las divide en grupos de size_group
    unique_categories = df[cat_col1].unique()
    if len(unique_categories) > size_group:
        num_plots = int(np.ceil(len(unique_categories) / size_group))

        for i in range(num_plots):
            # Selecciona un subconjunto de categorías para cada gráfico
            categories_subset = unique_categories[i * size_group:(i + 1) * size_group]
            data_subset = count_data[count_data[cat_col1].isin(categories_subset)]

            # Crea el gráfico
            plt.figure(figsize=(5, 3))
            ax = sns.barplot(x=cat_col1, y='count', hue=cat_col2, data=data_subset, order=categories_subset)

            # Añade títulos y etiquetas
            plt.title(f'Relación entre {cat_col1} y {cat_col2} - Grupo {i + 1}')
            plt.xlabel(cat_col1)
            plt.ylabel('Frecuencia' if relative_freq else 'Conteo')
            plt.xticks(rotation=45)

            # Mostrar valores en el gráfico
            if show_values:
                for p in ax.patches:
                    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                                ha='center', va='center', fontsize=10, color='black', xytext=(0, size_group),
                                textcoords='offset points')

            # Muestra el gráfico
            plt.show()
    else:
        # Crea el gráfico para menos de size_group categorías
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x=cat_col1, y='count', hue=cat_col2, data=count_data)

        # Añade títulos y etiquetas
        plt.title(f'Relación entre {cat_col1} y {cat_col2}')
        plt.xlabel(cat_col1)
        plt.ylabel('Frecuencia' if relative_freq else 'Conteo')
        plt.xticks(rotation=45)

        # Mostrar valores en el gráfico
        if show_values:
            for p in ax.patches:
                ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha='center', va='center', fontsize=10, color='black', xytext=(0, size_group),
                            textcoords='offset points')

        # Muestra el gráfico
        plt.show()

### 6.1.2. Análisis bivariante numérica-categórica

In [ ]:
def plot_categorical_numerical_relationship(df, categorical_col, numerical_col, show_values=False, measure='mean'):
    # Calcula la medida de tendencia central (mean o median)
    if measure == 'median':
        grouped_data = df.groupby(categorical_col)[numerical_col].median()
    else:
        # Por defecto, usa la media
        grouped_data = df.groupby(categorical_col)[numerical_col].mean()

    # Ordena los valores
    grouped_data = grouped_data.sort_values(ascending=False)

    # Si hay más de 5 categorías, las divide en grupos de 5
    if grouped_data.shape[0] > 5:
        unique_categories = grouped_data.index.unique()
        num_plots = int(np.ceil(len(unique_categories) / 5))

        for i in range(num_plots):
            # Selecciona un subconjunto de categorías para cada gráfico
            categories_subset = unique_categories[i * 5:(i + 1) * 5]
            data_subset = grouped_data.loc[categories_subset]

            # Crea el gráfico
            plt.figure(figsize=(10, 6))
            ax = sns.barplot(x=data_subset.index, y=data_subset.values)

            # Añade títulos y etiquetas
            plt.title(f'Relación entre {categorical_col} y {numerical_col} - Grupo {i + 1}')
            plt.xlabel(categorical_col)
            plt.ylabel(f'{measure.capitalize()} de {numerical_col}')
            plt.xticks(rotation=45)

            # Mostrar valores en el gráfico
            if show_values:
                for p in ax.patches:
                    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                                textcoords='offset points')

            # Muestra el gráfico
            plt.show()
    else:
        # Crea el gráfico para menos de 5 categorías
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x=grouped_data.index, y=grouped_data.values)

        # Añade títulos y etiquetas
        plt.title(f'Relación entre {categorical_col} y {numerical_col}')
        plt.xlabel(categorical_col)
        plt.ylabel(f'{measure.capitalize()} de {numerical_col}')
        plt.xticks(rotation=45)

        # Mostrar valores en el gráfico
        if show_values:
            for p in ax.patches:
                ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                            textcoords='offset points')

        # Muestra el gráfico
        plt.show()


def plot_combined_graphs(df, columns, whisker_width=1.5, bins = None):
    num_cols = len(columns)
    if num_cols:
        
        fig, axes = plt.subplots(num_cols, 2, figsize=(12, 5 * num_cols))
        print(axes.shape)

        for i, column in enumerate(columns):
            if df[column].dtype in ['int64', 'float64']:
                # Histograma y KDE
                sns.histplot(df[column], kde=True, ax=axes[i,0] if num_cols > 1 else axes[0], bins= "auto" if not bins else bins)
                if num_cols > 1:
                    axes[i,0].set_title(f'Histograma y KDE de {column}')
                else:
                    axes[0].set_title(f'Histograma y KDE de {column}')

                # Boxplot
                sns.boxplot(x=df[column], ax=axes[i,1] if num_cols > 1 else axes[1], whis=whisker_width)
                if num_cols > 1:
                    axes[i,1].set_title(f'Boxplot de {column}')
                else:
                    axes[1].set_title(f'Boxplot de {column}')

        plt.tight_layout()
        plt.show()

def plot_grouped_boxplots(df, cat_col, num_col):
    unique_cats = df[cat_col].unique()
    num_cats = len(unique_cats)
    group_size = 5

    for i in range(0, num_cats, group_size):
        subset_cats = unique_cats[i:i+group_size]
        subset_df = df[df[cat_col].isin(subset_cats)]
        
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=cat_col, y=num_col, data=subset_df)
        plt.title(f'Boxplots of {num_col} for {cat_col} (Group {i//group_size + 1})')
        plt.xticks(rotation=45)
        plt.show()



def plot_grouped_histograms(df, cat_col, num_col, group_size):
    unique_cats = df[cat_col].unique()
    num_cats = len(unique_cats)

    for i in range(0, num_cats, group_size):
        subset_cats = unique_cats[i:i+group_size]
        subset_df = df[df[cat_col].isin(subset_cats)]
        
        plt.figure(figsize=(10, 6))
        for cat in subset_cats:
            sns.histplot(subset_df[subset_df[cat_col] == cat][num_col], kde=True, label=str(cat))
        
        plt.title(f'Histograms of {num_col} for {cat_col} (Group {i//group_size + 1})')
        plt.xlabel(num_col)
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()


### 6.1.3. Análisis bivariante numéricas

In [ ]:
def grafico_dispersion_con_correlacion(df, columna_x, columna_y, tamano_puntos=50, mostrar_correlacion=False):
    """
    Crea un diagrama de dispersión entre dos columnas y opcionalmente muestra la correlación.

    Args:
    df (pandas.DataFrame): DataFrame que contiene los datos.
    columna_x (str): Nombre de la columna para el eje X.
    columna_y (str): Nombre de la columna para el eje Y.
    tamano_puntos (int, opcional): Tamaño de los puntos en el gráfico. Por defecto es 50.
    mostrar_correlacion (bool, opcional): Si es True, muestra la correlación en el gráfico. Por defecto es False.
    """

    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x=columna_x, y=columna_y, s=tamano_puntos)

    if mostrar_correlacion:
        correlacion = df[[columna_x, columna_y]].corr().iloc[0, 1]
        plt.title(f'Diagrama de Dispersión con Correlación: {correlacion:.2f}')
    else:
        plt.title('Diagrama de Dispersión')

    plt.xlabel(columna_x)
    plt.ylabel(columna_y)
    plt.grid(True)
    plt.show()

In [ ]:
# Relación entre variables numéricas
# Calculo el coef de correlación para ver si hay, si es positiva o negativa
# df[[var1,var2]].corr()

# Ahora calculo el p-valor que me dirá si aceptar mi hipótesis.
# Si me sale una posible correlación positiva calculo:
from scipy.stats import pearsonr

def pvalor_pos(df, var1, var2):
    corr, pvalor = pearsonr(df[var1], df[var2], alternative= "greater")
    print("Correlación: ", corr)
    print("P-valor:", pvalor)
    print("Si p-valor < 0.05 existe correlación positiva")


# Si me sale una posible correlación negativa calculo:
from scipy.stats import pearsonr

def pvalor_neg(df, var1, var2):
    corr, pvalor = pearsonr(df[var1], df[var2], alternative= "less")
    print("Correlación: ", corr)
    print("P-valor:", pvalor)
    print("Si p-valor < 0.05 existe correlación negativa")

Análisis multivariante completo en el archivo 10_ejercicios_analisis_multivariante_I

## 6.2. Relación variables categóricas

### 6.2.1. Relación categóricas. Chi-Cuadrado, test para extrapolar los resultados de una muestra a un total

In [ ]:
from scipy.stats import chi2_contingency

# Relación entre categórica-categórica
# el p-valor será la probabilidad de que la hipótesis nula (que hay independencia) sea cierta

def chi2(df, var1, var2):
    tabla_contingencia = pd.crosstab(df[var1], df[var2])
    chi2, p, dof, expected = chi2_contingency(tabla_contingencia)

    print("Valor Chi-Cuadrado:", chi2)
    print("P-Value:", p)
    print("Grados de Libertad:", dof)
    print("Tabla de Frecuencias Esperadas:\n", expected)
    print("Si el p-valor < 0.05 existe dependencia entre las variables")

### 6.2.2. Relación categórica-numérica

In [ ]:
# Relación categórica binaria-numérica (cuando el histograma se parece a una normal) T-TEST


# Relación categórica binaria-numérica (cuando el histograma NO se parece a una normal) PRUEBA U DE MANN-WHITNEY
# la hipótesis nula es que no hay diferencia estadística significativa entre las medianas de los valores 
from scipy.stats import mannwhitneyu

def pruebau(df, columna_binaria, columna_numerica):
    binaria = df[columna_binaria].unique()

    grupo_a = df.loc[df[columna_binaria] == binaria[0]][columna_numerica]
    grupo_b = df.loc[df[columna_binaria] == binaria[1]][columna_numerica]

    u_stat, p_valor = mannwhitneyu(grupo_a, grupo_b)

    print("Estadístico U:", u_stat)
    print("Valor p:", p_valor)
    print("Si p-valor < 0.05 existe relación entre las medianas de cada grupo de la binaria")


# Relación categórica no binaria-numérica. TEST ANOVA
# la hipótesis nula es que no hay relación estadística
from scipy import stats

def anova(df, columna_categorica, columna_numerica):
    grupos = df[columna_categorica].unique()
    agrupaciones = [df[df[columna_categorica] == grupo][columna_numerica] for grupo in grupos] 
    f_val, p_val = stats.f_oneway(*agrupaciones) # El método * separa todos los elementos de la lista y los pasa como argumento a la función
    print("Valor F:", f_val)
    print("Valor p:", p_val)
    print("Si p-valor < 0.05 existe relación entre la variable categórica y numérica, ya que se distribuyen distinto según los valores de la categórica")



## 6.3. Análisis multivariante

### 6.3.1. Análisis multivariante de categóricas

In [ ]:
# Análisis 3 categóricas
def analisis_multi_3categoricas(df, var1_target, var2, var3):
    # donde var1 se separa según valores, var2 es la que agrupa en eje x y var3 es la de la leyenda
    diccionario_multivariante = {}
    for valor in df[var1_target].unique():
        diccionario_multivariante[valor] = df.loc[df[var1_target] == valor,[var2,var3]] 

    for valor,df_datos in diccionario_multivariante.items():
        print(f"Respuesta {valor}:")
        plot_categorical_relationship_fin(df_datos,var2,var3, relative_freq= True, show_values= True, size_group=len(df[var2].unique()))


# Análisis 1 categórica 2 numéricas
def analisis_multi_1categorica_2numericas(df, var1_categorica, var2, var3):
    # donde var1 se separa según valores, var2 es la que agrupa en eje x y var3 es la de la leyenda
    diccionario_multivariante = {}
    for valor in df[var1_categorica].unique():
        diccionario_multivariante[valor] = df.loc[df[var1_categorica] == valor,[var2,var3]] 

    for valor,df_datos in diccionario_multivariante.items():
        print(f"Respuesta {valor}:")
        grafico_dispersion_con_correlacion(df_datos,var2,var3, tamano_puntos=20, mostrar_correlacion= True)


# Análisis 2 categóricas 1 numérica
def analisis_num_catcat(df, num_col, cat1, cat2):
    """
    Analiza la relación entre una variable numérica y dos categóricas.
    Muestra estadísticas, tabla resumen por grupos y un boxplot.
    
    Parámetros:
    df      -> dataframe
    num_col -> nombre de la columna numérica
    cat1    -> primera categórica (factor 1)
    cat2    -> segunda categórica (factor 2)
    """
    
    print("\n=== 1. Estadísticas por combinación de categorías ===")
    tabla = df.groupby([cat1, cat2])[num_col].agg(["count", "mean", "median", "std"])
    display(tabla)

    print("\n=== 2. Boxplot múltiple para visualizar diferencias ===")
    plt.figure(figsize=(10,5))
    sns.boxplot(data=df, x=cat1, y=num_col, hue=cat2)
    plt.title(f"{num_col} según {cat1} y {cat2}")
    plt.show()

    print("\n=== 3. Observaciones sugeridas ===")
    print(f"- Observa si {num_col} varía mucho entre categorías de {cat1}.")
    print(f"- Comprueba si dentro de cada {cat1}, el subgrupo {cat2} muestra diferencias claras.")
    print(f"- Fíjate en la dispersión (boxplot) y en si las medias del pivot table son diferentes.")

### 6.3.2. Análisis multivariante numéricas

In [ ]:
# Matriz de correlación de todas las variables numéricas
def matriz_correlacion(df):
    corr_matrix = df.corr(numeric_only= True) 
    plt.figure(figsize=(10, 8)) 
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", 
                cbar=True, square=True, linewidths=.5) # el cmap es el rango de colores usado para representar "el calor"

    plt.title('Matriz de Correlación')
    plt.xticks(rotation=45)  # Rota las etiquetas de las x si es necesario
    plt.yticks(rotation=45)  # Rota las etiquetas de las y si es necesario

    plt.show()

    return corr_matrix

# Diagramas de dispersión de varias numéricas
def diagramas_dispersion(df, lista_columnas_numericas):
    subset = df[lista_columnas_numericas]
    sns.pairplot(subset)
    plt.show()

